In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import requests
import time

In [2]:
url = "http://gbfs.citibikenyc.com/gbfs/gbfs.json"
data_urls = requests.get(url).json()

station_status_url =data_urls['data']['en']['feeds'][2]['url']
 
# posix_stamp = data_urls['last_updated']
# last_update = datetime.datetime.fromtimestamp(posix_stamp)
# print(f"Last update was: {last_update}") 

In [3]:
station_status = requests.get(station_status_url).json()

a_station = station_status['data']['stations'][0]
a_station['station_id']
a_station['last_reported']
a_station
a_station 

{'eightd_has_available_keys': False,
 'num_ebikes_available': 0,
 'num_docks_disabled': 0,
 'legacy_id': '72',
 'station_status': 'out_of_service',
 'last_reported': 86400,
 'num_docks_available': 0,
 'num_bikes_disabled': 0,
 'station_id': '72',
 'is_installed': 0,
 'is_returning': 0,
 'num_bikes_available': 0,
 'is_renting': 0}

# Piece out useful status from station

In [4]:


data_dict = {}
for station in station_status['data']['stations']:
    station_id = station['station_id']
    station_time = station['last_reported']
    
    subsubdict = {'num_bikes_available': station['num_bikes_available'], 
           'num_ebikes_available': station['num_ebikes_available'], 
            }
    subdict = {station_time: subsubdict}
    data_dict[station_id] = subdict 

In [5]:

for i in range(100):
    station_status = requests.get(station_status_url).json()
    
    for station in station_status['data']['stations']:
         
        station_id = station['station_id']
        station_time = station['last_reported']
        if station_time not in data_dict[station_id]:
            subsubdict = {'num_bikes_available': station['num_bikes_available'], 
                   'num_ebikes_available': station['num_ebikes_available'], 
                    }
            subdict = {station_time: subsubdict}
            olddict = data_dict[station_id]
            data_dict[station_id] = {**olddict, **subdict}
    time.sleep(1)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
data_dict

{'72': {86400: {'num_bikes_available': 0, 'num_ebikes_available': 0}},
 '79': {1597277792: {'num_bikes_available': 18, 'num_ebikes_available': 0}},
 '82': {1597277885: {'num_bikes_available': 16, 'num_ebikes_available': 0}},
 '83': {1597278038: {'num_bikes_available': 40, 'num_ebikes_available': 0}},
 '116': {1597277833: {'num_bikes_available': 17, 'num_ebikes_available': 0}},
 '119': {1597276120: {'num_bikes_available': 17, 'num_ebikes_available': 0}},
 '120': {1597277626: {'num_bikes_available': 1, 'num_ebikes_available': 0}},
 '127': {1597277976: {'num_bikes_available': 16, 'num_ebikes_available': 0},
  1597278146: {'num_bikes_available': 17, 'num_ebikes_available': 0}},
 '128': {1597278023: {'num_bikes_available': 24, 'num_ebikes_available': 0},
  1597278113: {'num_bikes_available': 25, 'num_ebikes_available': 0}},
 '143': {1597277427: {'num_bikes_available': 15, 'num_ebikes_available': 0}},
 '144': {1597277543: {'num_bikes_available': 43, 'num_ebikes_available': 0}},
 '146': {1597

In [7]:
for key, item in data_dict['247'].items():
    print(datetime.datetime.fromtimestamp(key) )

2020-08-12 20:19:56
2020-08-12 20:21:02
2020-08-12 20:22:08


In [14]:
with open("sample.json", "w") as outfile:  
    json.dump(data_dict, outfile) 

In [8]:
dict_of_df = {k: pd.DataFrame(v) for k,v in data_dict.items()}
df = pd.concat(dict_of_df, axis=1).transpose()
df

,,num_bikes_available,num_ebikes_available
72,86400,0,0
79,1597277792,18,0
82,1597277885,16,0
83,1597278038,40,0
116,1597277833,17,0
...,...,...,...
4109,86400,0,0
4112,86400,0,0
4113,86400,0,0
4114,86400,0,0


# dataframe merging example

### get first frame

In [9]:
# get frame
df = pd.DataFrame.from_dict(station_status['data']['stations'])

### get second frame later

In [ ]:
time.sleep(10)

In [12]:
station_status = requests.get(station_status_url).json()
df2 = pd.DataFrame.from_dict(station_status['data']['stations'])
df2

,num_bikes_disabled,last_reported,is_installed,station_id,legacy_id,is_renting,eightd_has_available_keys,num_ebikes_available,num_docks_disabled,is_returning,station_status,num_bikes_available,num_docks_available
0,0,86400,0,72,72,0,False,0,0,0,out_of_service,0,0
1,1,1597278191,1,79,79,1,False,0,0,1,active,17,15
2,0,1597277885,1,82,82,1,False,0,0,1,active,16,11
3,0,1597278038,1,83,83,1,False,0,0,1,active,40,22
4,1,1597277833,1,116,116,1,False,0,0,1,active,17,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,0,86400,0,4109,4109,0,False,0,0,0,out_of_service,0,0
1075,0,86400,0,4112,4112,0,False,0,0,0,out_of_service,0,0
1076,0,86400,0,4113,4113,0,False,0,0,0,out_of_service,0,0
1077,0,86400,0,4114,4114,0,False,0,0,0,out_of_service,0,0


### merge two frames

In [13]:
df_concat = pd.concat([df, df2]) 

unused_cols = ['num_docks_disabled', 'eightd_has_available_keys', 'is_renting', 'legacy_id', 'is_installed', 'num_bikes_disabled' , 'is_returning' ]
df_temp = df_concat.drop(labels=unused_cols, axis=1)
bool_active = df_concat['station_status'] == 'active'
df_temp2 = df_temp[bool_active].drop(labels='station_status', axis=1)

df_temp3 = df_temp2.drop_duplicates()

df4 = df_temp3.copy()
df4

,last_reported,num_docks_available,station_id,num_ebikes_available,num_bikes_available
1,1597277792,14,79,0,18
2,1597277885,11,82,0,16
3,1597278038,22,83,0,40
4,1597277833,32,116,0,17
5,1597276120,2,119,0,17
...,...,...,...,...,...
1002,1597278203,9,3997,0,11
1015,1597278178,3,4012,0,19
1023,1597278183,15,4020,1,7
1035,1597278183,17,4036,0,17
